In [1]:
file_name =  "nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001202_00000043.root"
event = 427785
subrun = 1979

In [2]:
# External packages
import sys
import warnings
warnings.filterwarnings("ignore") # suppress warnings
import awkward as ak 

# pyutils classes
from pyutils.pyprint import Print
from pyutils.pyprocess import Processor
from pyutils.pyselect import Select

processor = Processor(use_remote=True, location="disk")
data = processor.process_data(
    file_name=file_name,
    branches = { 
        "evt" : ["run", "subrun", "event"],
        "crv" : ["crvcoincs.time", "crvcoincs.nHits", "crvcoincs.pos.fCoordinates.fZ"],
        "trk" : ["trk.nhits", "trk.nactive", "trk.pdg", "trkqual.valid", "trkqual.result"],
        "trkhits" : ["trkhits"],
        "trkfit" : ["trksegs", "trksegpars_lh"],
        "trkmc" : ["trkmcsim"]
    }
)

[pyutils] ⭐️ Setting up...
[pyutils] ✅ Ready
[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = True
	location = disk
	schema = root
	verbosity=1
[pyprocess] ✅ Completed process on nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001202_00000043.root


In [3]:
this_event = data[ 
    (data["evt"]["subrun"] == subrun) & 
   (data["evt"]["event"] == event)
]

In [4]:
Print().print_n_events(this_event)

[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
evt.run: 1202
evt.subrun: 1979
evt.event: 427785
crv.crvcoincs.time: [555]
crv.crvcoincs.nHits: [23]
crv.crvcoincs.pos.fCoordinates.fZ: [1.63e+03]
trk.trk.nhits: [74, 23, 75, 23]
trk.trk.nactive: [65, 21, 65, 22]
trk.trk.pdg: [-11, 11, -13, 13]
trk.trkqual.valid: [True, True, True, True]
trk.trkqual.result: [0.99, 0.0246, 0.987, 0.0208]
trkhits.trkhits.plane: [[35, 35, 35, 35, 34, 34, 34, 33, 33, ..., 6, 6, 6, 0, 24, 24, 24, 24, 34], ...]
trkhits.trkhits.panel: [[0, 0, 5, 5, 1, 2, 2, 1, 1, 2, 2, ..., 1, 2, 2, 2, 2, 5, 0, 0, 0, 5, 1], ...]
trkhits.trkhits.layer: [[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, ..., 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], ...]
trkhits.trkhits.straw: [[73, 74, 22, 23, 27, 72, 73, 48, 49, ..., 0, 2, 3, 2, 10, 12, 13, 36, 24], ...]
trkhits.trkhits.state: [[1, 1, 1, -1, -1, 1, -1, 1, -1, 

In [5]:


def one_track_per_event(data):
    # Count unique pdgs
    pdgs = data["trk"]["trk.pdg"]
    n_unique_pdgs = ak.num(ak.run_lengths(ak.sort(pdgs)), axis=-1)
    n_total_pdgs = ak.num(pdgs, axis=-1)
    has_duplicate_pdgs = n_unique_pdgs < n_total_pdgs  # duplicates exist
    
    # Check nhits overlap
    nhits = data["trk"]["trk.nhits"]
    pairs = ak.combinations(nhits, 2, axis=-1)
    n1, n2 = ak.unzip(pairs)
    max_diff = ak.max(abs(n1 - n2), axis=1)  # per event
    similar_nhits = max_diff < 5  # within 5 hits

    data["similar_nhits"] = similar_nhits
    data["has_duplicate_pdgs"] = has_duplicate_pdgs

    # Require both
    one_track_per_event = ~has_duplicate_pdgs & similar_nhits 
    return one_track_per_event  

    
data["one_track_per_event"] = one_track_per_event(data)
# print("PDGs:", pdgs)
# print("nHits:", trk_nhits)
# print("Has duplicate PDGs:", has_duplicate_pdgs)
# print("Has different nhits:", has_different_nhits)
# print("Too many hypotheses:", too_many_hypotheses)
# print("One track per event:", one_track_per_event)

In [6]:
Print().print_n_events(data[(~data["has_duplicate_pdgs"])]["trk"], n_events=10)

[pyprint] ⭐️ Initialised Print with verbose = False and precision = 1
[pyprint] ⭐️ Printing 10 event(s)...

-------------------------------------------------------------------------------------
trk.nhits: [42, 42, 42, 42]
trk.nactive: [32, 42, 33, 42]
trk.pdg: [-11, 11, -13, 13]
trkqual.valid: [True, True, True, True]
trkqual.result: [0.0233, 0.0548, 0.0246, 0.0478]
-------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------
trk.nhits: [35, 35, 35, 35]
trk.nactive: [33, 32, 33, 32]
trk.pdg: [11, -11, 13, -13]
trkqual.valid: [True, True, True, True]
trkqual.result: [0.0336, 0.0271, 0.0261, 0.0212]
-------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------
trk.nhits: [43, 43, 43, 43]
trk.nactive: [34, 36, 34, 38]
trk.pdg: [11, -11, 13, -13]
trkqual.valid: [True, Tr

## Process the entire dataset and make a distribution of nhits for tracks with no duplicate pdgs

In [8]:
from pyutils.pyprocess import Skeleton

# Create your custom processor class
class CosmicProcessor(Skeleton):
    """Your custom file processor 
    
    This class inherits from the Skeleton base class, which provides the 
    basic structure and methods withing the Processor framework 
    """
    def __init__(self):
        """Initialise your processor with specific configuration
        
        This method sets up all the parameters needed for this specific analysis.
        """
        # Call the parent class's __init__ method first
        # This ensures we have all the base functionality properly set up
        super().__init__()

        # Now override parameters from the Skeleton with the ones we need
        # Data selection configuration 
        self.defname = "nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.root"
        # self.file_name = "nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001202_00024888.root"
        self.branches = { 
            "evt" : ["run", "subrun", "event"],
            "trk" : ["trk.nhits", "trk.pdg"],
        }
        self.use_remote = True     # Use remote file via mdh
        self.location = "disk"     # File location
        self.max_workers = 50       # Limit the number of workers
        self.verbosity = 2        # Set verbosity 
        self.worker_verbosity = 0    
        self.use_processes = True  # Use processes rather than threads
        

        # self.logger = Logger(
        #     print_prefix = "[CosmicProcessor]"
        # )
            
        # Toggle cuts OFF
        # self.inactive_cuts = []
            
        # Custom prefix for log messages from this processor
        # self.logger.log("Initialised", "success")
        
    # # ==========================================
    # # Define the core processing logic
    # # ==========================================
    # # This method overrides the parent class's process_file method
    # # It will be called automatically for each file by the execute method
    def process_file(self, file_name): 
        """Process a single ROOT file
        
        This method will be called for each file in our list.
        It extracts data, processes it, and returns a result.
        
        Args:
            file_name: Path to the ROOT file to process
            
        Returns:
            A tuple containing the histogram (counts and bin edges)
        """
        try:
            # Create a local pyprocess Processor to extract data from this file
            # This uses the configuration parameters from our class
            this_processor = Processor(
                use_remote=self.use_remote,     # Use remote file via mdh
                location=self.location,         # File location
                verbosity=self.worker_verbosity # self.verbosity        # Reduce output in worker threads
            )
            
            # Extract the data 
            this_data = this_processor.process_data(
                file_name = file_name, 
                branches = self.branches
            )
            
            # ---- Analysis ----
            
            # results = self.analyse.execute(this_data, file_name)

            return this_data 
        
        except Exception as e:
            # Handle any errors that occur during processing
            self.logger.log(f"Error processing {file_name}: {e}", "success")
            return None

# Create an instance of our custom processor
cosmic_processor = CosmicProcessor()

[Skeleton] ⭐️ Skeleton init


In [ ]:
results = cosmic_processor.execute()

[Skeleton] ⭐️ Starting analysis
[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = True
	location = disk
	schema = root
	verbosity=2
[pyprocess] 👀 Loading file list for SAM definition: nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.root
[pyprocess] ✅ Successfully loaded file list
	SAM definition: nts.mu2e.CosmicCRYSignalAllOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.root
	Count: 822 files
[pyprocess] ⭐️ Starting processing on 822 files with 50 processes


Processing:   0%|                              | 0/822 [00:00<?, ?file/s]

In [11]:
results

<Array [{evt: {...}, trk: {...}}, ..., {...}] type='3219 * {evt: {run: int3...'>

In [ ]:
pdgs = data["trk"]["trk.pdg"]

has_single_e_minus = ak.sum(pdgs == 11, axis=-1) <= 1
has_single_e_plus = ak.sum(pdgs == -11, axis=-1) <= 1
has_single_mu_plus = ak.sum(pdgs == 13, axis=-1) <= 1
has_single_mu_minus = ak.sum(pdgs == -13, axis=-1) <= 1

print(pdgs)

print(has_single_e_minus)
print(has_single_e_plus)
print(has_single_mu_plus)
print(has_single_mu_minus)

single_track = has_single_e_minus & has_single_e_plus & has_single_mu_plus & has_single_mu_minus

print(single_track)

In [ ]:
Print().print_n_events(data[single_track]["trk"], 10)

In [ ]:
Print().print_n_events(data[~single_track]["evt"], 4)